In [2]:
import numpy as np
import pandas as pd
import h5py
import pysam

In [3]:
fasta_ref = pysam.FastaFile('../reference/hg38.genome.fa')

In [4]:
peaks_df = pd.read_csv('../ENCSR000EGM/data/origp.bed', sep='\t', header=None, names=['chrom', 'st', 'en', 'name', 'score', 'strand', 'signalValue', 'p', 'q', 'summit'])

In [5]:
chr1_peaks_df = peaks_df[peaks_df['chrom'].isin(('chr1',))]
chr1_peaks_df = chr1_peaks_df.reset_index(drop=True)

In [6]:
chr1_peaks_df.head()

,chrom,st,en,name,score,strand,signalValue,p,q,summit
0,chr1,35641559,35641799,.,1000,.,6.52180,-1.0,0.00080,120
1,chr1,27600557,27600797,.,599,.,6.59866,-1.0,0.00209,120
2,chr1,1944232,1944472,.,1000,.,6.69948,-1.0,0.01882,120
3,chr1,159925706,159925946,.,598,.,6.81884,-1.0,0.03458,120
4,chr1,7954373,7954613,.,579,.,6.88965,-1.0,0.04601,120


In [7]:
chr1_peaks_df['start'] = chr1_peaks_df['st'] + chr1_peaks_df['summit'] - (2114 // 2)
chr1_peaks_df['end'] = chr1_peaks_df['st'] + chr1_peaks_df['summit'] + (2114 // 2)

In [8]:
num_peaks = chr1_peaks_df.shape[0]
print(num_peaks)

6085


In [9]:
chr1_peaks_df.head()
print(type(chr1_peaks_df))

<class 'pandas.core.frame.DataFrame'>


In [10]:
seq1 = fasta_ref.fetch('chr1', 35640622, 35642736).upper()

In [11]:
def insert_variant(seq, allele, position):
    left, right = seq[:position-1], seq[position:]
    return left + allele + right

In [12]:
seq1_var = insert_variant(seq1, 'G', 1057)
print(seq1_var, len(seq1_var))

AGCGACACCAGGAAACTTATCAGAGATTCGGGGATCTAGAGAGCCAGTCCCACAGTCTCACTCTCCTCAGGATCCCACCTTATGCCGCCAACAAACTAACCTCATTACTAGCGACACCAGGAAACTTATCAGAGATTCGGGGATCTAGAGAGCCAGGCTCTGGGCAGGCGCTTAAACATTTATAAACTGATTCCCTTTAGGCAACTCTCACCAAGCAAGAAGCATTTAGCGAAGGCCAGAGCAGAGCACTGACCCAGGAGCCCTGGAGGAGCAAATGCCAAGGTCCCAGACCTGTCTGCCTGTTTCGCCTGTAATCCCAGCACTTTGGAAGGCCGAGACGGGCGGATCTCCTGAGGTCGGGAGTTCGAGACCAGCCTGACCAATATGGAGAAATCCACATCTCTACTAATAATACAAAAATTAGCCAGGCGTGGTGGCGCATATCTGTTCCCAGCTACTCGGGAGGCTGAGGCACGAGAATCGCTTGAACCCGGGAGACAAAGACTGTGCTAAGCCGAGATCGCGCCATTGCACTCCGGCCTGGACAACAAGAGCGAAACCCCTCTCGAAAAAATAAATAAATAAATAAAATAAGTACGGGCAGGGCAGGCCGGCTTCCATCTCTCAGATCCTCCCTGGTACTTATTCAACCCCCGACAAACTCCTTCTGGCCGCTCCTACACCCCAGGCCTGGCCGGGCCTCCCCTGCTTCCTCTCACCGTCCTTCATCTGGACAATATTGCTGGCGGCCACCCGGTCGGAGGCGACAAGAACATAGTCGGGGCCTTGGATACCGATGAGGTACTCCATGGTGGCGGAAGGCCAGGGGCTGCAGGTCCGACACAGCACGAGACTCGCCCGCTTCCAGGTCTCACCGGTGAGACAGCACCTCAGAGCGAAGATTGGCGCGACGCCTGCAGCACGACTTCCACGGCGCTCTCGGATGACGTACAACTGTCGCGAGAGGTTGCAAAGCGGGCGCGGCGCCGGGTGCCTTATG

In [13]:
def add_variant(df, chromosome, position, allele):
    df2 = pd.DataFrame([[chromosome, position, allele]], \
        columns = ['chrom', 'st', 'allele'])
    return df.append(df2)

In [14]:
var_df = pd.DataFrame(columns = ['chrom', 'st', 'allele'])
var_df = add_variant(var_df, 'chr11', 86103988, 'G')
var_df = add_variant(var_df, 'chr11', 86103988, 'A')
var_df.head()

,chrom,st,allele
0,chr11,86103988,G
0,chr11,86103988,A


In [15]:
var_df['summit'] = 0
var_df['signalValue'] = 10
var_df.head()

,chrom,st,allele,summit,signalValue
0,chr11,86103988,G,0,10
0,chr11,86103988,A,0,10


In [16]:
var_df.to_csv('rs1237999.bed', sep='\t', encoding='utf-8', header=False, index=False)